<a href="https://colab.research.google.com/github/hansong0219/Advanced-DeepLearning-Study/blob/master/GAN/CGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conditional GAN

DCGAN 에서는 이미지가 임의로 생성되며, 생성기에서 어떤 숫자를 생성할지는 제어할 수 없다. 이 문제를 원-핫 벡터를 추가로 입력을 받는 것으로 해결한 것이 바로 Conditional GAN (CGAN) 이다. 

동일한 GAN의 구조를 사용하면서도 생성기에서 원-핫 벡터를 추가로 받고 새로운 Dense 계층을 추가해 잠재벡터와 연결된다. 또, 판별기에서는 새로운 Dense 계층이 추가된다. 이 새로운 게층은 원-핫 벡터를 처리하고 뒤따라나오는 CNN 계층의 다른 입력과 연결하기에 적합한 형상으로 변경하기 위해 사용된다.

생성기는 100차원 입력 벡터와 특정 숫자로 부터 가짜 이미지를 생성하는 것을 학습하며, 판별기는 진짜와 가짜 이미지, 그리고 그에 대응하는 레이블을 기반으로 진짜와 가짜 이미지를 분류한다.

CGAN 의 기본 원리는 GAN 과 동일하지만,  판별기와 생성기 입력에 원-핫 레이블 y 라는 조건이 부여된다는 점이 다르다.

판별기와 생성기는 아래와 같이 구성된다.

## GPU 할당

Colab 이 아닌 windows 환경에서는 아래의 코드를 실행하여 gpu의 할당을 구현할 수 있다. 

In [25]:
#import tensorflow as tf 
#physical_devices =tf.config.experimental.list_pysical_devices('GPU')
#tf.config.exprimental.set_memory_growth(physical_devices[0],True)

## Module 불러오기

In [26]:
from tensorflow.keras.layers import Activation, Dense, Input
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.layers import Reshape, Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

import numpy as np
import math
import matplotlib.pyplot as plt
import os
import argparse

# Model 빌드

# 판별기 구성 함수 정의

In [27]:
def build_discriminator(inputs, y_labels, image_size):
  # 판별기 모델 구성
  """
  입력은 Dense 계층 다음에서 연결되며, 진짜와 가짜를 판별하기 위한 LeakyReLU-Conv2D 스택으로 구성되어있다. 
  """

  # 입력 인자 
  """
  inputs (layer) : 판별기의 입력계층
  y_labels (layer) : 입력에 조건을 부여하는 원-핫 벡터를 위한 입력 계층
  image_size : 한변의 목표 크기 (이미지는 정사각형으로 가정한다)
  """

  #return - Model : 판별기의 모델을 반환한다.

  kernel_size = 5
  layer_filters = [32, 64, 128, 256]

  x = inputs
  y = Dense(image_size*image_size)(y_labels)
  y = Reshape((image_size,image_size, 1))(y)
  x = concatenate([x,y])

  for filters in layer_filters:
    # 첫 3개 합성곱 계층에서는 strides = 2 이며, 마지막 계층은 strides = 1 을 사용한다.
    if filters == layer_filters[-1]:
      strides = 1
    else:
      strides = 2

    x = LeakyReLU(alpha = 0.2)(x)
    x = Conv2D(filters = filters, kernel_size = kernel_size, strides= strides, padding='same')(x)

  
  x = Flatten()(x)
  x = Dense(1)(x)
  x = Activation('sigmoid')(x)

  # 입력은 y_labels 에 의해 조건이 부여됨 
  discriminator = Model([inputs, y_labels], x, name='discriminator')

  return discriminator

# 생성기 모델 구성 함수 정의

In [28]:
def build_generator(inputs, y_labels, image_size):
  # 생성기 모델 구성
  """
  입력은 Dense 계층 전에 연결된다. 
  가짜 이미지를 생성하기 위한 BN-ReLU-Conv2DTranspose 스택, 출력계층의 활성화 함수로 sigmoid를 사용함
  """

  # 함수 입력 인수 
  """
  inputs (layer): 생성기의 입력 계층 (노이즈 벡터)
  y_labels (layer): 입력에 조건을 부여하는 원-핫 벡터 입력계층
  image_size : 한변의 목표 크기 
  """
  
  #return - Model : 생성기 모델 

  image_resize = image_size//4

  #네트워크 매개 변수 
  kernel_size = 5
  layer_filters = [128, 64, 32, 1]

  x = concatenate([inputs, y_labels], axis=1)
  x = Dense(image_resize * image_resize * layer_filters[0])(x)
  x = Reshape((image_resize, image_resize, layer_filters[0]))(x)

  for filters in layer_filters:
    # 처음 두 합성곱 계층은 strides = 2를 사용
    # 마지막 두 계층은 strides = 1을 사용
    if filters > layer_filters[-2]:
      strides = 2
    else:
      strides = 1

    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(filters=filters, kernel_size=kernel_size, strides=strides, padding='same')(x)

  
  x = Activation('sigmoid')(x)

  # 입력은 y_label에 의해 조건이 부여됨
  generator = Model([inputs, y_labels], x, name='generator')

  return generator

# Data Set 전처리

In [29]:
# MNIST 불러오기 
(x_train, y_train), (_, _) = mnist.load_data()

In [30]:
# CNN 을 위한 데이터형상 조정 및 정규화
image_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_train = x_train.astype('float32')/255

# 라벨 지정
num_labels = np.amax(y_train) + 1
y_train = to_categorical(y_train)

# CGAN 모델 구성

In [31]:
model_name = "cgan_mnist"

# 네트워크 매개 변수 지정
latent_size = 100
batch_size = 64
train_steps = 40000
lr = 2e-4
decay = 6e-8
input_shape = (image_size, image_size, 1)
label_shape = (num_labels, )

In [32]:
# 판별기 구성
inputs = Input(shape=input_shape, name='discriminator_input')
labels = Input(shape=label_shape, name='class_labels')

discriminator = build_discriminator(inputs, labels, image_size)
optimizer = RMSprop(lr=lr, decay=decay)

discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

discriminator.summary()

Model: "discriminator"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
class_labels (InputLayer)       [(None, 10)]         0                                            
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 784)          8624        class_labels[0][0]               
__________________________________________________________________________________________________
discriminator_input (InputLayer [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
reshape_4 (Reshape)             (None, 28, 28, 1)    0           dense_5[0][0]                    
______________________________________________________________________________________

In [33]:
# 생성기 구성
input_shape = (latent_size, )
inputs = Input(shape=input_shape, name='z_input')
generator = build_generator(inputs, labels, image_size)
generator.summary()

Model: "generator"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
z_input (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
class_labels (InputLayer)       [(None, 10)]         0                                            
__________________________________________________________________________________________________
concatenate_5 (Concatenate)     (None, 110)          0           z_input[0][0]                    
                                                                 class_labels[0][0]               
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 6272)         696192      concatenate_5[0][0]      

In [34]:
# 적대적 모델 구성
optimizer = RMSprop(lr=lr*0.5, decay=decay*0.5)

# 적대적 모델의 훈련에서는 판별기에 가중치를 반영하지 않는다
discriminator.trainable = False
    
outputs = discriminator([generator([inputs, labels]), labels])
adversarial = Model([inputs, labels], outputs, name=model_name)

adversarial.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
adversarial.summary()

Model: "cgan_mnist"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
z_input (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
class_labels (InputLayer)       [(None, 10)]         0                                            
__________________________________________________________________________________________________
generator (Functional)          (None, 28, 28, 1)    1364225     z_input[0][0]                    
                                                                 class_labels[0][0]               
__________________________________________________________________________________________________
discriminator (Functional)      (None, 1)            1090001     generator[0][0]         

모델 변수 입력

In [35]:
models = (generator, discriminator, adversarial)
data = (x_train, y_train)
params = (batch_size, latent_size, train_steps, num_labels, model_name)

# Plot 함수정의

In [36]:
def plot_images(generator,
                noise_input,
                noise_class,
                show=False,
                step=0,
                model_name="gan"):

    os.makedirs(model_name, exist_ok=True)
    filename = os.path.join(model_name, "%05d.png" % step)
    images = generator.predict([noise_input, noise_class])
    print(model_name , " labels for generated images: ", np.argmax(noise_class, axis=1))
    plt.figure(figsize=(2.2, 2.2))
    num_images = images.shape[0]
    image_size = images.shape[1]
    rows = int(math.sqrt(noise_input.shape[0]))
    for i in range(num_images):
        plt.subplot(rows, rows, i + 1)
        image = np.reshape(images[i], [image_size, image_size])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.savefig(filename)
    if show:
        plt.show()
    else:
        plt.close('all')


# 모델 훈련

모델 훈련 함수 정의

In [49]:
def train(models, data, params):
  # 판별기와 적대적 네트워크의 훈련 
  """
  배치 단위로 판별기와 적대적 네트워크가 교대로 훈련된다. 판별기는 우선 적절한 레이블을 붙인 진짜와 가짜 이미지를 사용하여 운련된다.
  그 다음으로 적대적 네트워크는 진짜인 척 하는 가짜 이미지를 사용하여 훈련이 된다. 

  판별기의 입력은 진짜 이미지에 대한 훈련 라벨과 가짜 이미지에 대한 랜덤 레이블에 의해 조건이 부여된다. 
  적대적 네트워크의 입력은 랜덤 레이블에 의해 조건이 부여된다.
  """

  # 함수 입력 인수
  """
  model (list) : Generator, Discriminator, Adversarial models
  data (list): x_train, y_train 데이터
  params (list): 네트워크 매개 변수
  """

  # GAN 모델 
  generator, discriminator, adversarial = models
  # 이미지 레이블 
  x_train, y_train = data  
  # 네트워크 매개변수
  batch_size, latent_size, train_steps, num_labels, model_name = params

  # 생성기의 이미지는 500 epochs 마다 저장됨
  save_interval = 500
  # 훈련하는 동안 생성기 출력을 보여주기 위한 노이즈 벡터
  noise_input = np.random.uniform(-1.0, 1.0, size=[16,latent_size])

  #노이즈에 조건을 부여할 원-핫 레이블
  noise_class = np.eye(num_labels)[np.arange(0, 16) % num_labels]
  
  # 훈련 데이터 세트의 요소 개수 
  train_size = x_train.shape[0]

  print(model_name, "Labels for generated images:", np.argmax(noise_class,axis=1))

  for i in range(train_steps):
    # 1 배치 에 대한 판별기 훈련 
    
    # 데이터 세트에서 진짜 이미지를 임의로 선정
    rand_indices = np.random.randint(0, train_size, size=batch_size)
    real_images = x_train[rand_indices]
    # 대응하는 원-핫 레이블 생성 
    real_labels = y_train[rand_indices]

    #생성기를 사용해 노이즈로 부터 가짜 이미지 생성
    noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
    
    #임의의 원-핫 레이블 할당
    fake_labels = np.eye(num_labels)[np.random.choice(num_labels, batch_size)]
    #가짜레이블 조건으로 하는 가짜이미지 생성
    fake_images = generator.predict([noise, fake_labels])

    #진짜 이미지 + 가짜이미지 를 1배치 훈련데이터로 사용
    x = np.concatenate((real_images, fake_images))
    y_labels = np.concatenate((real_labels, fake_labels))

    #진짜와 가짜 이미지에 레이블을 붙임
    #진짜이미지 레이블은 1, 가짜 이미지 레이블은 0
    y = np.ones([2*batch_size,1])
    y[batch_size:,:] = 0.0

    #log 기록 
    loss, acc = discriminator.train_on_batch([x, y_labels],y)
    log = "%d: [discriminator loss: %f, acc:%f]" %(i, loss, acc)

    #1 배치에 대한 적대적 네트워크 훈련
    #레이블이 1인 가짜 원핫 레이블을 조건으로 하는 가짜 이미지의 1배치
    #적대적 네으퉈크에서 파별기의 가중치는 고정되기 때문에 생성기만 훈련이된다. 

    #균등분포를 통해 노이즈 생성 
    noise = np.random.uniform(-1.0, 1.0, size=[batch_size,latent_size])

    #임의의 원-핫 레이블 할당
    fake_labels = np.eye(num_labels)[np.random.choice(num_labels,batch_size)]

    # 가짜 이미지에 진짜로 속이는 레이블을 붙임
    y = np.ones([batch_size, 1])
    
    #적대적 네으퉈크 훈련
    #판별기 훈련과 달리 변수에 가짜 이미지를 별도의 변수로 저장하지 않으며, 분류를 위해 가짜 이미지는 적대적 네트워크의 판별기 입력으로 전달됨

    #log 기록
    loss, acc = adversarial.train_on_batch([noise, fake_labels], y)
    log = "%s [adversarial loss : %f, acc : %f]" %(log, loss, acc)
    print(log)

    if (i+1) % save_interval == 0:
      if (i+1) == train_steps:
        show = True
      else:
        show = False

      plot_images(generator, noise_input=noise_input,noise_class=noise_class,show=show, step=(i+1), model_name=model_name) 

  generator.save(model_name+".h5")

In [50]:
train(models, data, params)

cgan_mnist Labels for generated images: [0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5]
0: [discriminator loss: 0.614064, acc:0.828125] [adversarial loss : 1.154150, acc : 0.000000]
1: [discriminator loss: 0.493613, acc:1.000000] [adversarial loss : 1.486888, acc : 0.000000]
2: [discriminator loss: 0.347265, acc:1.000000] [adversarial loss : 0.965978, acc : 0.000000]
3: [discriminator loss: 0.206373, acc:1.000000] [adversarial loss : 2.836246, acc : 0.000000]
4: [discriminator loss: 0.203657, acc:0.976562] [adversarial loss : 0.236512, acc : 1.000000]
5: [discriminator loss: 0.140484, acc:1.000000] [adversarial loss : 2.231820, acc : 0.000000]
6: [discriminator loss: 0.085524, acc:1.000000] [adversarial loss : 0.942631, acc : 0.000000]
7: [discriminator loss: 0.036926, acc:1.000000] [adversarial loss : 0.601828, acc : 1.000000]
8: [discriminator loss: 0.025494, acc:1.000000] [adversarial loss : 0.462506, acc : 1.000000]
9: [discriminator loss: 0.019973, acc:1.000000] [adversarial loss : 0.342037, ac

KeyboardInterrupt: ignored

# 생성기 테스트

In [22]:
def test_generator(generator, class_label=None):
    noise_input = np.random.uniform(-1.0, 1.0, size=[16, 100])
    step = 0
    if class_label is None:
        num_labels = 10
        noise_class = np.eye(num_labels)[np.random.choice(num_labels, 16)]
    else:
        noise_class = np.zeros((16, 10))
        noise_class[:,class_label] = 1
        step = class_label

    plot_images(generator,
                noise_input=noise_input,
                noise_class=noise_class,
                show=True,
                step=step,
                model_name="test_outputs")


In [ ]:
class_label = None
test_generator(generator, class_label)